# Scalarization testing


### ASF

Now there is standard Achievement Scalarizing Function implemented, so let's use it.

In [2]:
from ASF import ASF
from gradutil import *
from pyomo.opt import SolverFactory
import logging

In [3]:
logger = logging.getLogger('pyomo.opt')
logger.setLevel(logging.ERROR)

In [4]:
revenue, carbon, deadwood, ha = init_boreal()
ide = ideal(False)
nad = nadir(False)
opt = SolverFactory('cplex')

In [5]:
ind = 100
x = np.dstack((new_normalize(nan_to_bau(revenue[:ind]).values),
               new_normalize(nan_to_bau(carbon[:ind]).values),
               new_normalize(nan_to_bau(deadwood[:ind]).values),
               new_normalize(nan_to_bau(ha[:ind]).values)))
X = np.dstack((nan_to_bau(revenue[:ind]).values,
               nan_to_bau(carbon[:ind]).values,
               nan_to_bau(deadwood[:ind]).values,
               nan_to_bau(ha[:ind]).values))

In [6]:
%%time
ref = np.array((ide[0], 0, 0, 0))
asf = ASF(ide, nad, ref, X)
opt.solve(asf.model)
print(np.sum(values_to_list(asf.model, X), axis=0))

In [7]:
%%time
ref = np.array((0, ide[1], 0, 0))
asf = ASF(ide, nad, ref, X)
opt.solve(asf.model)
print(np.sum(values_to_list(asf.model,X), axis=0))

In [8]:
%%time
ref = np.array((0, 0, ide[2], 0))
asf = ASF(ide, nad, ref, X)
opt.solve(asf.model)
print(np.sum(values_to_list(asf.model, X), axis=0))

### Timing

The ASF function seems to work properly so no problem. Let's see how much time does it take when running in full speed.

In [9]:
%%time
n_revenue = nan_to_bau(revenue)
n_carbon = nan_to_bau(carbon)
n_deadwood = nan_to_bau(deadwood)
n_ha= nan_to_bau(ha)
x_full = np.dstack((new_normalize(n_revenue.values),
                    new_normalize(n_carbon.values),
                    new_normalize(n_deadwood.values),
                    new_normalize(n_ha.values)))
X_full = np.dstack((n_revenue.values,
                    n_carbon.values,
                    n_deadwood.values,
                    n_ha.values))

In [10]:
from interactiveBoreal import ReferenceFrame as rf
kehys = rf()

In [11]:
extreme1 = np.array([2.49578348e+08, 2.82311284e+06, 8.17647344e+04, 1.20426387e+04])

In [12]:
extreme2 = np.array([8.73864412e+07, 4.42779337e+06, 2.02829180e+05, 1.81784694e+04])

In [13]:
extreme3 = np.array([1.13414218e+08, 4.19114380e+06, 2.12316813e+05, 1.90109867e+04])

In [16]:
extreme4 = np.array([1.44157068e+08, 3.91362164e+06, 2.05681872e+05, 1.99452821e+04])

In [ ]:
%%time
asf_fullx1 = ASF(ide, nad, extreme1, X_full)
opt.solve(asf_fullx1.model)

In [30]:
solution1 = np.sum(values_to_list(asf_fullx1.model,X_full), axis=0)

In [ ]:
%%time
asf_fullx2 = ASF(ide, nad, extreme2, X_full)
opt.solve(asf_fullx2.model)

In [27]:
solution2 = np.sum(values_to_list(asf_fullx2.model,X_full), axis=0)

In [ ]:
%%time
asf_fullx3 = ASF(ide, nad, extreme3, X_full)
opt.solve(asf_fullx3.model)

In [28]:
solution3 = np.sum(values_to_list(asf_fullx3.model,X_full), axis=0)

In [ ]:
%%time
asf_fullx4 = ASF(ide, nad, extreme4, X_full)
opt.solve(asf_fullx4.model)

In [29]:
solution4 = np.sum(values_to_list(asf_fullx4.model,X_full), axis=0)

In [53]:
for e,s in zip([extreme1,extreme2,extreme3,extreme4], [solution1,solution2,solution3,solution4]):
    '''if (e<s).all():
        print('Dominated')
    elif (s<e).all():
        print('Dominating')
    else:
        print('Non-dominating')'''
    print('{} & {} \\\\'.format('Surrogate Front', ' & '.join(['{:,.0f}'.format(value).replace(',', ' ') for value in e])))
    print('{} & {} \\\\'.format('Real Front', ' & '.join(['{:,.0f}'.format(value).replace(',', ' ') for value in s])))

## Comparing scalarizations

Now we have ASF, STOM ja GUESS scalarizations available. Lets compare them

In [21]:
ref_test = np.array((2,3,4,2))

In [22]:
asf = ASF(ide, nad, ref_test, x, scalarization='ASF')
opt.solve(asf.model)
print(np.sum(values_to_list(asf.model, X), axis=0))

In [23]:
stom = ASF(ide, nad, ref_test, x, scalarization='STOM')
opt.solve(stom.model)
print(np.sum(values_to_list(stom.model, X), axis=0))

In [24]:
guess = ASF(ide, nad, ref_test, x, scalarization='GUESS')
opt.solve(guess.model)
print(np.sum(values_to_list(guess.model, X), axis=0))

There truly is differences between results, so it seems that scalarization functions are working properly (?)

In [25]:
    from time import time
    from datetime import timedelta
    import logging
    import interactiveBoreal as ib

    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    start = time()
    logger.info('Started')
    logger.info('Initializing...')
    kehys = ib.ReferenceFrame()
    logger.info('Initialized. Time since start {}'.
                format(timedelta(seconds=int(time()-start))))
    nclust = 600
    seedn = 6
    '''
    logger.info('Clustering...')
    import simplejson as json
    with open('clusterings/new_300.json', 'r') as file:
        clustering = json.load(file)
    kehys.xtoc = np.array(clustering['5']['xtoc'])
    kehys.weights = np.array([sum(kehys.xtoc == i)
                              for i in range(nclust)
                              if sum(kehys.xtoc == i) > 0])
    
    kehys.cluster(nclust=nclust, seedn=seedn)
    logger.info('Clustered. Time since start {}'.
                format(timedelta(seconds=int(time()-start))))
    '''
    kyle1 = np.array((1858.0, 58.69, 2.651, 0.27))
    init_ref = kyle1 * 68700
    # init_ref = np.array((0, 0, kehys.ideal[2], 0))
    ref = kehys.normalize_ref(init_ref)

    logger.info('Using ideal: {} and nadir: {}'.
                format(kehys.ideal, kehys.nadir))
    logger.info('Reference point: {}.'.format(init_ref))
    logger.info('Solving...')

    '''
    data = kehys.centers
    nvar = len(kehys.x_norm)
    weights = kehys.weights/nvar
    '''
    data = kehys.x_norm_stack
    nvar = len(kehys.x_norm)
    weights = np.ones(nvar)/nvar

    ''' Because everything is scaled, scale these too'''
    ideal = kehys.normalize_ref(kehys.ideal)
    nadir = kehys.normalize_ref(kehys.nadir)

    solver_name = 'cplex'

    asf = ASF(ideal, nadir, ref, data, weights=weights, nvar=nvar,
              scalarization='asf')

    asf_solver = ib.Solver(asf.model, solver=solver_name)
    asf_solver.solve()
    logger.info('Solved')

In [ ]:
asf_values = kehys.values(kehys.x_stack, weights, model=asf.model)
asf_values

In [ ]:
asf_values/68700

In [ ]:
init_ref

In [ ]:
kehys.ideal, init_ref, kehys.nadir

In [ ]:
ideal, ref, nadir
